This notebook is designed to find optimal segmentation parameters for a series of images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, core, io, plot, transforms
from natsort import natsorted
from pathlib import Path
from tqdm import trange

# Check GPU availability
if not core.use_gpu():
    raise ImportError("No GPU access detected.")

model = models.CellposeModel(gpu = True)

These are the parameters that will need to be modified across each image.

`FLOW_THRESHOLD` is the maximum allowed error of the flows for each mask. Default == 0.4.
* **Increase** this threshold if there are not as many masks as you’d expect (or turn off completely with 0.0)
* **Decrease** this threshold if there are too many ill-shaped masks.

`CELLPROB_THRESHOLD` determines proability that a detected object is a cell. Default == 0.0.
* **Increase** this threshold if there are too many masks, especiallly from dull/dim areas.
* **Decrease** this threshold if there are not as many masks as you’d expect or if masks are too small

`NITER` is the number of iterations for dynamics involved in mask creation. A default value of 0 means it is proportional to diameter. If you have long ROIs, set to a larger number like 2000.

`TILE_NORM_BLOCKSIZE` determines the size of blocks used for normalizing the image. The default is 0, which means the entire image is normalized together. You may want to change this to 100-200 pixels if you have inhomogeneous brightness across your image.

`BATCH_SIZE` is number of images loaded and run through the model at the same time. For example, if you set `batch_size = 8`, it will take 8 images, stack them into a batch, and send them through the network together.

`CHANNELS` is used to select which channels you want to use in the segmentation. This can be a phase, brightfield, or nuclear stain etc. When using more than 1 channel, order does not matter.

In [ ]:
# Parameters to change!

# Directory containing images to process
IMAGE_DIR = "/vast/scratch/users/moore.z/projects/incu/test_anu/"
IMAGE_EXTENSION = ".tiff"  # Change this to your image file extension

# Segmentation parameters
FLOW_THRESHOLD = 0
CELLPROB_THRESHOLD = -1
NITER = 5000
TILE_NORM_BLOCKSIZE = 500
BATCH_SIZE = 8

# Channel selection (modify this list as needed)
CHANNELS = [0]  # List of channel indices to use, e.g., [0], [0, 1], [0, 1, 2], etc.

In [ ]:
# Setup directory path
image_dir = Path(IMAGE_DIR)
if not image_dir.exists():
    raise FileNotFoundError(f"Directory does not exist: {IMAGE_DIR}")

# Find all image files (excluding masks and flows)
image_files = natsorted([
    f for f in image_dir.glob(f"*{IMAGE_EXTENSION}") 
    if "_masks" not in f.name and "_flows" not in f.name
])

if not image_files:
    raise FileNotFoundError(f"No {IMAGE_EXTENSION} files found in {IMAGE_DIR}")

print(f"Found {len(image_files)} image files to process")

print("\n" + "="*50)
print("TESTING ON SINGLE IMAGE")
print("="*50)

# Load and preprocess test image using last image
test_idx = len(image_files) - 1
if test_idx >= len(image_files):
    test_idx = 0
    
# test_file = image_files[test_idx]
test_file = image_files[0]
print(f"Loading test image: {test_file.name}")

# Load and preprocess image
img = io.imread(test_file)
img = transforms.move_min_dim(img, force=True)
img = transforms.normalize_img(img, normalize=True, percentile=(1, 50))

print(f"Image shape: {img.shape}")
print(f"Assuming channel dimension is last with {img.shape[-1]} channels")

# Select channels for processing
print(f"Using channels: {CHANNELS}")

# Validate channel indices
max_channel = img.shape[-1] - 1
for ch in CHANNELS:
    if ch > max_channel:
        raise ValueError(f"Channel index {ch} exceeds available channels (0-{max_channel})")

# Select channels directly
img_processed = img[..., CHANNELS]
print(f"Processed image shape: {img_processed.shape}")

# Run segmentation on test image
print("Running segmentation on test image...")
masks, flows, styles = model.eval(
    img_processed, 
    batch_size = BATCH_SIZE,
    flow_threshold = FLOW_THRESHOLD,
    cellprob_threshold = CELLPROB_THRESHOLD,
    normalize = {"tile_norm_blocksize": TILE_NORM_BLOCKSIZE},
    niter = NITER
)

print(f"Segmentation complete. Found {len(np.unique(masks))-1} cells")

# Visualize results
fig = plt.figure(figsize = (30, 12))
plot.show_segmentation(fig, img_processed, masks, flows[0])
plt.tight_layout()
plt.show()

In [ ]:
# Change this to True to process all images within the folder
if False:
    print("\n" + "="*50)
    print("BATCH PROCESSING ALL IMAGES")
    print("="*50)
    
    # Process all images
    for i in trange(len(image_files), desc="Processing images"):
        file_path = image_files[i]
        
        # Load image
        img = io.imread(file_path)
        
        # Run segmentation
        masks, flows, styles = model.eval(
            img,
            batch_size=BATCH_SIZE,
            flow_threshold=FLOW_THRESHOLD,
            cellprob_threshold=CELLPROB_THRESHOLD,
            normalize={"tile_norm_blocksize": TILE_NORM_BLOCKSIZE},
            niter=NITER
        )
        
        # Save results
        output_path = image_dir / f"{file_path.stem}_masks{IMAGE_EXTENSION}"
        io.imsave(output_path, masks)
        
        # Save ROIs (regions of interest)
        roi_path = image_dir / f"{file_path.stem}_rois.zip"
        io.save_rois(masks, roi_path)
    
    print(f"\nProcessing complete! Results saved to: {image_dir}")
    print(f"Mask files: *_masks{IMAGE_EXTENSION}")
    print(f"ROI files: *_rois.zip")